# Creating mapping metrix

In [ ]:
# !pip install geopandas matplotlib
# pip install h5py

In [10]:
import geopandas as gpd
import pandas as pd
import zipfile
import os
import h5py

In [3]:
# Find the shapefile (.shp) within the extracted folder
shapefile_path = None
for root, dirs, files in os.walk("input_data/shapefile_data"):
    for file in files:
        if file.endswith(".shp"):
            shapefile_path = os.path.join(root, file)
            break

# Load the shapefile using GeoPandas and inspect its content
if shapefile_path:
    gdf = gpd.read_file(shapefile_path)
    print("Shapefile loaded successfully")

    # Display the first few rows of the GeoDataFrame and check its structure
    print(gdf.head())
    print(gdf.info())
    print("Geometry column unique values:", gdf.geometry.unique())
else:
    print("Shapefile not found in the extracted data.")


Shapefile loaded successfully
    PCON10CD                  PCON10NM PCON10NMW   PCON24CD  \
0  E14000530                 Aldershot      None  E14001063   
1  E14000844      North East Hampshire      None  E14001063   
2  E14000531       Aldridge-Brownhills      None  E14001064   
3  E14001012             Walsall South      None  E14001064   
4  E14000532  Altrincham and Sale West      None  E14001065   

                   PCON24NM PCON24NMW  ObjectId geometry  
0                 Aldershot      None         1     None  
1                 Aldershot      None         2     None  
2       Aldridge-Brownhills      None         3     None  
3       Aldridge-Brownhills      None         4     None  
4  Altrincham and Sale West      None         5     None  
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1407 entries, 0 to 1406
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   PCON10CD   1407 non-null   object  


In [ ]:
# # Number of unique values in PCON10CD and PCON24CD
# unique_pcon10cd = gdf['PCON10CD'].nunique()
# unique_pcon24cd = gdf['PCON24CD'].nunique()

# print("Number of unique values in PCON10CD:", unique_pcon10cd)
# print("Number of unique values in PCON24CD:", unique_pcon24cd)

Number of unique values in PCON10CD: 650
Number of unique values in PCON24CD: 650


In [5]:
file_path = "input_data/population_by_age.csv"
pop = pd.read_csv(file_path)
pop.head()

,con_code,con_name,rn_code,rn_name,nat_code,nat_name,age,con_number,con_pc,rn_number,rn_pc,nat_number,nat_pc,data_source
0,E14001063,Aldershot,E12000008,South East,K04000001,England and Wales,0 to 17,24886,0.208677,1962885.0,0.209267,12506535,0.207619,Mid-2022 estimate
1,E14001063,Aldershot,E12000008,South East,K04000001,England and Wales,18 to 24,9094,0.076256,714721.0,0.076198,5008522,0.083146,Mid-2022 estimate
2,E14001063,Aldershot,E12000008,South East,K04000001,England and Wales,25 to 34,18455,0.154751,1147461.0,0.122333,8098257,0.134438,Mid-2022 estimate
3,E14001063,Aldershot,E12000008,South East,K04000001,England and Wales,35 to 49,25101,0.210480,1842715.0,0.196455,11575752,0.192167,Mid-2022 estimate
4,E14001063,Aldershot,E12000008,South East,K04000001,England and Wales,50 to 64,22246,0.186540,1865056.0,0.198837,11745365,0.194983,Mid-2022 estimate


In [6]:
con_code_sums = pop.groupby('con_code')['con_number'].sum().reset_index()
con_code_sums.head()

,con_code,con_number
0,E14001063,119256
1,E14001064,93720
2,E14001065,101272
3,E14001066,90793
4,E14001067,98053


In [ ]:
# # Number of unique 'con_code' values
# unique_con_codes = pop['con_code'].nunique()
# print("Number of unique con_code:", unique_con_codes)

Number of unique con_code: 650


In [ ]:
# # Check for NaN values in the 'con_number' column
# nan_count = merged_df['con_number'].isna().sum()
# print("Number of NaN values in con_number:", nan_count)

Number of NaN values in con_number: 0


In [8]:
merged_df = gdf.merge(con_code_sums, how='left', left_on='PCON24CD', right_on='con_code')
merged_df = merged_df.rename(columns={'con_number': 'pop_2022'})
merged_df = merged_df[['PCON10CD', 'PCON24CD', 'pop_2022']].rename(columns={
    'PCON10CD': 'code_2010',
    'PCON24CD': 'code_2024'
})

# Assuming merged_df has the columns 'code_2010', 'code_2024', and 'pop_2022'
merged_df = merged_df.sort_values(by=['code_2010', 'code_2024']).reset_index(drop=True)

# Step 1: Calculate total population for each 'code_2010'
total_pop_2010 = merged_df.groupby('code_2010')['pop_2022'].sum().reset_index()
total_pop_2010.columns = ['code_2010', 'total_pop_2010']

# Step 2: Merge this total population back into the original DataFrame
merged_df = merged_df.merge(total_pop_2010, on='code_2010')

# Step 3: Calculate the normalized weight for each (code_2010, code_2024) pair
merged_df['weight'] = merged_df['pop_2022'] / merged_df['total_pop_2010']

# Step 4: Pivot to create the mapping matrix
mapping_matrix = merged_df.pivot_table(index='code_2010', columns='code_2024', values='weight', fill_value=0)
mapping_matrix = mapping_matrix.sort_index(axis=0).sort_index(axis=1)

# Display the mapping matrix
mapping_matrix.head()

code_2024,E14001063,E14001064,E14001065,E14001066,E14001067,E14001068,E14001069,E14001070,E14001071,E14001072,...,W07000103,W07000104,W07000105,W07000106,W07000107,W07000108,W07000109,W07000110,W07000111,W07000112
code_2010,,,,,,,,,,,,,,,,,,,,,
E14000530,1.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
E14000531,0.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
E14000532,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
E14000533,0.0,0.0,0.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
E14000534,0.0,0.0,0.0,0.0,0.242905,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
mapping_matrix.to_csv('input_data/constituencies_mapping_2010_2024.csv', index=False)

In [11]:
with h5py.File('input_data/constituencies_mapping_2010_2024.h5', 'w') as hf:
    # Save the DataFrame to the HDF5 file
    hf.create_dataset('df', data=mapping_matrix.values)
    # Optionally, save column names as metadata
    hf.attrs['columns'] = list(mapping_matrix.columns)

# Checks

In [12]:
# Step 1: Find a code_2010 that maps to multiple code_2024
example_code_2010 = merged_df['code_2010'].value_counts().idxmax()  # Finds the code_2010 with the most mappings

# Step 2: Filter merged_df for this example code_2010
example_mappings = merged_df[merged_df['code_2010'] == example_code_2010]

# Display the example mappings
print("Example code_2010:", example_code_2010)
print("Mappings to code_2024 with their pop_2022:")
print(example_mappings[['code_2024', 'pop_2022']])

# Step 3: Check the weights in the mapping matrix for this code_2010
print("\nWeights in mapping_matrix for code_2024 mapping to", example_code_2010, ":")
print(mapping_matrix.loc[example_code_2010, example_mappings['code_2024']].to_frame())

Example code_2010: S14000029
Mappings to code_2024 with their pop_2022:
      code_2024  pop_2022
1252  S14000084     99687
1253  S14000085    109113
1254  S14000086    107141
1255  S14000087     91460
1256  S14000088     96239

Weights in mapping_matrix for code_2024 mapping to S14000029 :
           S14000029
code_2024           
S14000084   0.197933
S14000085   0.216649
S14000086   0.212733
S14000087   0.181598
S14000088   0.191087


In [13]:
# Step 1: Find a code_2010 that maps to exactly one code_2024
single_mapping_code_2010 = merged_df['code_2010'].value_counts()[merged_df['code_2010'].value_counts() == 1].index[0]

# Step 2: Filter merged_df for this code_2010 with a single mapping
single_mapping_example = merged_df[merged_df['code_2010'] == single_mapping_code_2010]

# Display the single mapping
print("Single mapping code_2010:", single_mapping_code_2010)
print("Maps to code_2024 with pop_2022:")
print(single_mapping_example[['code_2024', 'pop_2022']])

# Step 3: Check the weight in the mapping matrix for this code_2010
print("\nWeight in mapping_matrix for code_2024 mapping to", single_mapping_code_2010, ":")
print(mapping_matrix.loc[single_mapping_code_2010, single_mapping_example['code_2024']].to_frame())


Single mapping code_2010: N06000013
Maps to code_2024 with pop_2022:
      code_2024  pop_2022
1185  N05000013     97983

Weight in mapping_matrix for code_2024 mapping to N06000013 :
           N06000013
code_2024           
N05000013        1.0
